# Washing Machine

---

## 목적: 세탁기에서 생성된 Log Data 분석

## 데이터 셋 정보 (Metadata)

- **washing_machine.csv**: 세탁기에서 생성된 Log 데이터
- **wm_metadata.csv**: 각 컬럼 내용에 대한 설명

---

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline 

# Pandas option
#pd.set_option('max_rows', 30)
#pd.set_option('max_columns', 15)

# 시각화 스타일 설정
# plt.style.use('fivethirtyeight') 
# sns.set_style('whitegrid')

#import warnings
#warnings.filterwarnings('ignore')

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 8
rcParams['axes.grid'] = True

# 1. 데이터 탐색

### 1.1 CSV 파일 읽어오기

In [ ]:
pd.read_csv('data/wm_metadata.csv')

In [ ]:
df = pd.read_csv('data/washing_machine.csv')
df

### 1.2 데이터 살펴보기

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.columns

### 1.3 임의 데이터 생성

> 분석을 위한 임의 변수들을 기존 데이터 프레임에 추가해보자.
>
> 1. 세제량 : 1 ~ 5 임의 생성
> 2. 세탁기 소음 : 40 ~ 70 임의 생성

In [ ]:
new_df = pd.DataFrame({'Detergent' : np.random.randint(1, 6, size=len(df)),
                       'NoiseLevel': np.random.randint(40, 71, size=len(df))})
new_df.head()

### 1.4 DataFrame 결합

In [ ]:
df = pd.concat([df, new_df], axis=1)
df

In [ ]:
df.columns

In [ ]:
df.shape

### 1.5 컬럼 삭제

In [ ]:
df.drop(['category_code', 'model_name', 'regist_country'], axis=1, inplace=True)

In [ ]:
df.shape

In [ ]:
df.columns

> **Q. 'FreshCare', 'TurboShot', 'InitialBit' 컬럼 제거, DataFrame 확인**

In [ ]:
df.drop(['FreshCare', 'TurboShot', 'InitialBit'], axis=1, inplace=True)

In [ ]:
df.shape

In [ ]:
df.columns

### 1.6 Grouping

- **로그 발생 유형 확인**

In [ ]:
df['event_type']

In [ ]:
df['event_type'].unique()

In [ ]:
df['event_type'].value_counts()

### 1.7 일시중지('WM_PAUSE') Log가 가장 많은 기계의 device_id 찾기

In [ ]:
df[df['event_type'] == 'WM_PAUSE']

In [ ]:
df[df['event_type'] == 'WM_PAUSE']['device_id']

> **Q. 일시중지(WM_PAUSE)인 device_id를 Grouping 해서 개수를 세어보자**

In [ ]:
df[df['event_type'] == 'WM_PAUSE']['device_id'].value_counts()

> **Q. 어느 기계(device_id)에 가장 많은 에러('WM_ERROR') Log가 기록되어 있을까?**

In [ ]:
df[df['event_type'] == 'WM_ERROR']['device_id'].value_counts()

### 1.8 Log가 많이 생성된 시간대

####  데이터 타입 확인: 'create_dt_utc'

In [ ]:
df['create_dt_utc'].head()

In [ ]:
df['create_dt_utc'][0]

In [ ]:
type(df['create_dt_utc'][0])

#### 데이터 타입 변경: String --> Datetime

In [ ]:
df['create_dt_utc'] = pd.to_datetime(df['create_dt_utc'])

In [ ]:
df['create_dt_utc']

In [ ]:
df['create_dt_utc'][0]

In [ ]:
type(df['create_dt_utc'][0])

#### 4시간 단위로 Grouping

In [ ]:
grouper = pd.Grouper(key='create_dt_utc', freq='4h')
grouper

In [ ]:
gp_4h = df.groupby(grouper)

In [ ]:
gp_4h.count()

In [ ]:
freq = gp_4h['create_dt_utc'].count()
freq

In [ ]:
freq.plot(kind='bar', figsize=(15, 7))

#### local time으로 변경: Series의 Index를 변경

In [ ]:
freq.tz_localize('UTC').tz_convert('Asia/Seoul').plot(kind='barh', figsize=(8, 8))

#### local time으로 변경: DataFrame의 컬럼을 변경

In [ ]:
import time
from datetime import timedelta

In [ ]:
# 컴퓨터에 설정된 타임존과 UTC의 차이 (단위: 초) (UTC - Current Timezone)
time.timezone

In [ ]:
df['create_dt_utc'] - timedelta(seconds=time.timezone)

> **Q. 한국 시각으로 변경한 데이터를 create_dt_kst 라는 새로운 컬럼에 넣어보자**

In [ ]:
df['create_dt_kst'] =  df['create_dt_utc'] - timedelta(seconds=time.timezone)
df

> **Q. 한국 시각으로 변경된 데이터로 그래프 그리기**

In [ ]:
grouper_kst = pd.Grouper(key='create_dt_kst', freq='4h')

In [ ]:
freq_kst = df.groupby(grouper_kst)['create_dt_kst'].count()

In [ ]:
freq_kst.plot(kind='barh', figsize=(8, 8))

In [ ]:
# end of file